In [2]:
from braindecode.models.shallow_fbcsp import ShallowFBCSPNet
from torch import nn
import torch
from braindecode.torch_ext.util import set_random_seeds
from braindecode.datautil.signal_target import SignalAndTarget
from braindecode.torch_ext.optimizers import AdamW
import torch.nn.functional as F
import numpy as np
import sys
sys.path.append('/home/mshaban/DeployedProjects/ac')
from src.utils import ensure_dir, shuffle_in_unison
import pickle
data_folder = "/home/mshaban/DeployedProjects/ac/data/"
model_output_path = data_folder+'models/eegbci.pkl'
ensure_dir(model_output_path)
labeled_data = pickle.load(open(data_folder+"labeled_data.pkl", "rb"))

### Create training_data, test_data

In [3]:
from sklearn.model_selection import train_test_split
X, y = [], []
min_n_times = 23040
event_id = []
e_cnt = 0
for event, trials in labeled_data.items():
    for it, trial in enumerate(trials):
        if trial[0].get_data().shape[1] > min_n_times:
            X.append(trial[0].get_data()[:, -1 * min_n_times:])
            # y.append(trial[1])
            eid = ''.join(map(str, trial[1]))
            v, a = trial[1]
            if v <= 5 and a <= 5:
                y.append(0)
            elif v > 5 >= a:
                y.append(1)
            elif v <= 5 < a:
                y.append(2)
            elif v >= 5 and a >= 5:
                y.append(3)
            # if eid in event_id:
            #     y.append(event_id.index(eid))
            # else:
            #     y.append(e_cnt)
            #     event_id.append(eid)
            #     e_cnt += 1


X, y = shuffle_in_unison(np.array(X).astype(np.float32), np.array(y).astype(np.int64))

In [4]:
train_set = SignalAndTarget(X[:40], y[:40])
valid_set = SignalAndTarget(X[40:55], y[40:55])
test_set = SignalAndTarget(X[55:], y[55:])
cuda = True
set_random_seeds(seed=20170629, cuda=cuda)
n_classes = 4
in_chans = train_set.X.shape[1]
crop_size = train_set.X.shape[2] // 10
model = ShallowFBCSPNet(in_chans=in_chans, n_classes=n_classes,
                        input_time_length=None,
                        final_conv_length=12)

if cuda:
    model.cuda()
# optimizer = AdamW(model.parameters(), lr=1*0.01, weight_decay=0.5*0.001) # these are good values for the deep model
optimizer = AdamW(model.parameters(), lr=0.0625 * 0.01, weight_decay=0)
model.compile(loss=F.nll_loss, optimizer=optimizer, iterator_seed=1, cropped=True)

# super_crop size
input_time_length = 23040

model.fit(train_set.X, train_set.y, epochs=100, batch_size=64, scheduler='cosine',
          validation_data=(valid_set.X, valid_set.y), input_time_length=input_time_length)

In [6]:
# print(model.epochs_df)
print(model.evaluate(test_set.X, test_set.y))
print(model.predict(test_set.X), test_set.y)

# with open(model_output_path, 'wb') as f:
#     pickle.dump(model, f)

{'misclass': 0.6666666666666667, 'runtime': 0.0010809898376464844, 'loss': 4.299388885498047}
[3 3 2] [1 3 3]
